Consider the 2DOF robotic structure from the figure below, for which $L1=L2=1 \;m$, $m1=m2=1\;kg$. The q1 and q2 initial conditions are $\pi$/4 and 1.
  
* Find the forward kinematics model using the D-H convention and create the robot using the robotics toolbox library DHRobot.
* Compute the C, D, and G matrices.    
* Implement in Python the robot model using the state space representation, with [odeintw](https://github.com/WarrenWeckesser/odeintw) (wrapper of odeint for working with matrices) for solving the MIMO diferential equations. Give two sine waves as the input joint torques.  
* Plot the positions and velocities compared with the inputs. Interpret the results.

<img src="../artwork/DynMod/2DOFstud.png" width=50% />

In [8]:
!pip install odeintw

In [18]:
%reset -f
import numpy as np
import matplotlib.pyplot as plt
import roboticstoolbox as rtb
import math as m
from roboticstoolbox import *
from spatialmath import *
from spatialmath.base import * 
from lab_functions import lim_prismatic
from odeintw import *
from scipy.interpolate import *
from scipy.integrate import odeint

##########  2C, 2D ############

pr_lim = 10

def model(x, t, u):
    # Reshape input for matrix operations
    u = np.array(u).reshape(2, 1)
    
    # Extract positions and velocities
    q = x[:2]  # Positions
    dq = x[2:]  # Velocities
    
    # Apply prismatic joint limit
    lim_prismatic(q, dq, pr_lim)
    
    # Define the robot's physical parameters
    m1 = 1  # Mass of link 1
    m2 = 1  # Mass of link 2
    l1 = 1  # Length of link 1
    l2 = 1  # Length of link 2
    r1 = l1 / 2  # Distance from joint axis to center of mass of link 1
    r2 = l2 / 2  # Distance from joint axis to center of mass of link 2
    I1 = m1 * l1**2 / 12  # Moment of inertia of link 1
    I2 = m2 * l2**2 / 12  # Moment of inertia of link 2
    g = 9.81  # Acceleration due to gravity
    
    # Calculate the inertia matrix D(q)
    D = np.array([[I1 + m2*l1**2 + 2*m2*l1*r2*np.cos(q[1]), I2 + m2*l1*r2*np.cos(q[1])],
                  [I2 + m2*l1*r2*np.cos(q[1]), I2]])
    
    # Calculate the Coriolis matrix C(q, dq)
    C = np.array([[-m2*l1*r2*np.sin(q[1])*dq[1], -m2*l1*r2*np.sin(q[1])*(dq[0]+dq[1])],
                  [m2*l1*r2*np.sin(q[1])*dq[0], 0]])
    
    # Calculate the gravity vector G(q)
    G = np.array([[m1*g*r1*np.sin(q[0]) + m2*g*(l1*np.sin(q[0]) + r2*np.sin(q[0]+q[1]))],
                  [m2*g*r2*np.sin(q[0]+q[1])]])
    
    # Inverse of inertia matrix
    D_inv = np.linalg.inv(D)
    
    # Compute the system acceleration (ddq)
    ddq = D_inv @ (u - C @ dq.reshape(2, 1) - G)
    
    # Combine the velocities and accelerations to form the state derivative
    xdot = np.concatenate((dq.reshape(2, 1), ddq), axis=0)
    
    return xdot.flatten()

# Initial conditions
x0 = np.array([0, 0, 0, 0])

n = 100 # Number of time points
t = np.linspace(0, 10, n) # Time points

# Control inputs as sine waves
u1 = np.sin(t)
u2 = np.sin(2*t)
u = np.vstack((u1, u2)).T

# Create interpolation functions for the control inputs
u_func = interp1d(t, u.T, kind='cubic', fill_value="extrapolate")

# Integrate the model over time using the interpolation function for u
x = odeint(model, x0, t, args=(u_func,))

# Extract positions and velocities
q = x[:, :2]
dq = x[:, 2:]

# Plot results0
plt.figure()

plt.subplot(3, 1, 1)
plt.plot(t, u[:, 0], 'g:', label='u1(t)')
plt.plot(t, u[:, 1], 'g--', label='u2(t)')
plt.ylabel('Control Input')
plt.legend(loc='best')

plt.subplot(3, 1, 2)
plt.plot(t, q[:, 0], 'b-', label='q1(t)')
plt.plot(t, q[:, 1], 'b--', label='q2(t)')
plt.ylabel('Positions')
plt.legend(loc='best')

plt.subplot(3, 1, 3)
plt.plot(t, dq[:, 0], 'r-', label='dq1(t)')
plt.plot(t, dq[:, 1], 'r--', label='dq2(t)')
plt.ylabel('Velocities')
plt.xlabel('Time')
plt.legend(loc='best')

plt.tight_layout()
plt.show()

ValueError: cannot reshape array of size 1 into shape (2,1)

In [12]:
############### animation 
q = q.reshape(n,1)
l = 1
robot = DHRobot([ RevoluteMDH(alpha=m.pi/2, offset=m.pi/2), RevoluteMDH(d=l, alpha=m.pi/2), ], base=SE3(0.6, 0, 0), name = '2DOF robot')
zers  = np.zeros_like(q)
coord = np.concatenate((q,zers),axis=1)
robot.plot(coord, movie='1dof.gif', dt=tf/n, shadow=False)

NameError: name 'q' is not defined